# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below to import the dataset

In [ ]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_dataset.csv")
df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           9.47              5.97         7.36           10.17       6.84   
1          10.05              8.84         9.76            8.38      10.15   
2          10.59             10.71        10.84           10.97       9.03   
3          11.00              8.44         8.32            9.65       7.87   
4          12.12             13.44        10.35            9.95      11.09   

   free sulfur dioxide  total sulfur dioxide  density  sulphates  alcohol  \
0                 9.15                  9.78     9.52      10.34     8.80   
1                 6.91                  9.70     9.01       9.23     8.80   
2                10.42                 11.46    11.25      11.34     9.06   
3                10.92                  6.97    11.07      10.66     8.89   
4                 9.38                 10.22     9.04       7.68    11.38   

   quality rating  
0               6  
1               7  
2               4  
3               8  
4               3

- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [ ]:
df['quality rating'].unique()

array([ 6,  7,  4,  8,  3,  1,  2, 10,  5,  9])

In [ ]:
df['quality rating'].value_counts()

10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: quality rating, dtype: int64

❓ Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [ ]:
y = df['quality rating'].map(lambda x: 0 if x < 6 else 1)

❓ Check the class balance of the new binary target

In [ ]:
y.value_counts()

0    50010
1    49990
Name: quality rating, dtype: int64

❓ Create your `X` by normalising the features. This will allow for fair comparison of different solvers.

In [ ]:
# Select only the features 
X = df.drop(columns=['quality rating'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Fit scaler to features
scaler = MinMaxScaler().fit(X)

# Scale features
X_scaled = scaler.transform(X)

# keep column names
X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns

## 2. LogisticRegression solvers

❓ Logistic Regression models can be optimized using different **solvers**. Make a comparison of the available solvers':
- Fit time - which solver is **the fastest**?
- Precision - **how different** are their respective precision scores?

Available solvers for Logistic Regression are `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this Stack Overflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

# List solver types to loop over
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

# Initiate scores and fit times lists to store for each model
scores = []
fit_times = []

# Loop over solvers
for solver in solvers:
    
    # Cross validate each model
    cv_log_s = cross_validate(
        LogisticRegression(solver=solver),
        X_scaled, y,
        cv = 5,
        scoring = ['precision']
    )

    # Append mean score and mean fit time to lists
    scores.append(cv_log_s['test_precision'].mean())
    fit_times.append(cv_log_s['fit_time'].mean())
    
# Create dataframe with each model's performance
solvers_performance = pd.DataFrame({"precision score":scores, "fit time": fit_times}, index = solvers)
solvers_performance

precision score  fit time
newton-cg         0.874386  0.460770
lbfgs             0.874389  0.450857
liblinear         0.874449  0.086412
sag               0.874352  0.220132
saga              0.874386  0.321405

In [ ]:
# YOUR ANSWER
fastest_solver = ""

In [ ]:
fastest_solver = solvers_performance['fit time'].idxmin()
fastest_solver

'liblinear'

<details>
    <summary>ℹ️ Click here for our interpretation</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function.

**The wine dataset**
    
If you check feature importance with sklearn's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> on the current dataset, you'll see many features result in almost 0 importance. Liblinear solver successively moves only along *one* direction at a time, regularizing the others with L1 regularization (a.k.a, setting their beta to 0), which might provide a good fit for a dataset where many features are not that important in predicting the target.

❗️There is a cost to searching for the best solver. Sticking with the default (`lbfgs`) may save the most time overall, sklearn provides you this grid for an idea of which solver to choose to start off with: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>



</details> 

###  🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/olivergiles/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/olivergiles/code/lewagon_dev/data-solutions/05-ML/04-Under-the-hood/02-Solvers/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

❓ Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(loss="log_loss")

cv_sgd = cross_validate(
    sgd_model,
    X_scaled, y,
    cv = 5,
    scoring = 'precision'
)

In [ ]:
# Precision Score
cv_sgd['test_score'].mean()

0.8785496907910229

In [ ]:
# Training time
cv_sgd['fit_time'].mean()

0.076513671875

☝️ The SGD model should have one of the shortest times (maybe even shorter than `liblinear`), for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single row as opposed to loading 100k rows into memory at a time.

## 4. Predictions

❓ Use the best model (balanced with short fit time and high precision) to predict the binary quality (0 or 1) of the following wine. Store your:
- `predicted_class`
- `predicted_proba_of_class` (i.e if your model predicted a class of 1 what is the probability it believes 1 to be the class should be between 0 and 1)

In [ ]:
new_wine = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_new_wine.csv')
new_wine

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           9.54              13.5        12.35            8.78      14.72   

   free sulfur dioxide  total sulfur dioxide  density  sulphates  alcohol  
0                 9.06                  9.67    10.15      11.17    12.17

In [ ]:
# Train model with fastest training time
best_model = SGDClassifier(loss="log_loss").fit(X_scaled,y)

# Scale new data using original scaler and restore the column names
new_X = pd.DataFrame(scaler.transform(new_wine))
new_X.columns = new_wine.columns

# Predict!
best_model.predict(new_X)

array([0])

In [ ]:
predicted_class = best_model.predict(new_X)[0]
predicted_class

0

In [ ]:
predicted_proba_of_class = best_model.predict_proba(new_X)[0, predicted_class]
predicted_proba_of_class

0.9669785275482368

# 🏁  Check your code and push your notebook

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/pavelliser/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/pavelliser/Desktop/DataSciProgram/data-solutions/05-ML/04-Under-the-hood/02-Solvers
plugins: anyio-3.4.0, dash-2.0.0
collecting ... collected 2 items

tests/test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
tests/test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.10s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master
